In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def f(X):
    return ((5*X[0]-X[1])**4 + (X[0]-2)**2 + X[0] - 2*X[1] + 12)

$
\text{Input: } \alpha,\text{ } \beta,\text{ } γ,\text{ } ϵ, \text{ }f : C → R \text{ where } C ⊆ R^n \text{ , n+1 starting points }
$


In [ ]:
def simplex_search(alpha, beta, gamma, epsilon, function, starting_points):

  # initialize the coefficients, function, starting points
  r ,c, e, epsilon, f, X = alpha, beta, gamma, epsilon, function, starting_points

  # number of iterations
  i = 0

  # create lists to keep track of the newly created variables

  X_bar_list = []
  X_h_list = []
  X_l_list = []
  X_new_list = []
  f_new_list = []
  type_list = []


  while (True):
    f_values = [f(X.iloc[i, :]) for i in range(len(X))]
    h_indx = np.argmax(f_values, axis=0)
    l_indx = np.argmin(f_values, axis=0)

    # argmax{f(xi)}
    xh = X.iloc[h_indx, :]
    # argmin{f(xi)}
    xl = X.iloc[l_indx, :]
    # x_bar = 1/n * sum xi when i != h
    x_bar = X.drop(h_indx, axis=0).mean()

    # start keeping track
    X_h_list.append((xh[0], xh[1]))
    X_l_list.append((xl[0], xl[1]))
    X_bar_list.append((x_bar[0],x_bar[1]))


    # * reflection *

    xr = x_bar + r * (x_bar - xh) # alpha (r) > 0, is the reflection coefficient
    if f(xl) > f(xr):

      # * expansion *

      xe = x_bar + e * (xr - x_bar) # gamma (e) > 1, is the expension coefficient

      if f(xr) > f(xe):

        # replace xh by xe, a new simplex is obtained
        X.iloc[h_indx, :] = xe
        type_list.append("E")
        X_new_list.append(xe)
        f_new_list.append(f(xe))


      else:
          # expansion is failed, replace xh by xr to obtain a new simplex
          X.iloc[h_indx, :] = xr
          type_list.append('R')
          X_new_list.append(xr)
          f_new_list.append(f(xr))

    else:

        # max i = 0, 1, 2, ..., n; i != h {fxi}
        max_index = np.argmax([f(X.drop(h_indx, axis=0).reset_index(drop=True).iloc[i, :]) for i in range(len(X) - 1)], axis=0)
        if (f(X.iloc[max_index, :]) > f(xr)):
          # replace xh by xr to obtain a new simplex
          X.iloc[h_indx, :] = xr
          type_list.append('R')
          X_new_list.append(xr)
          f_new_list.append(f(xr))


        else:
          # *contraction*
          if f(xh) > f(xr):
            # replacing xh by xr would make xr new xh
            xh_new = xr
          else:
            xh_new = xh
          xc = x_bar + c * (xh_new - x_bar) # 0 < beta (c) < 1 is the contraction coefficient


          if(f(xc) <= f(xh_new)):
            # replace xh by xc
            X.iloc[h_indx, :] = xc
            type_list.append('C')
            X_new_list.append(xc)
            f_new_list.append(f(xc))


          else:
              # xi = xi + 1/2 * (xl - xi)
              X = X + (0.5) * (xl - X)
              type_list.append('C')
              X_new_list.append([X.iloc[i, :] for i in range(len(X))])
              f_new_list.append([f(X.iloc[i, :]) for i in range(len(X))])


    i += 1
    # if stopping condition is met, stop the while block
    # sqrt of {i / (n+1) * sum (f(xi) - f(x_bar))^2}    < epsilon
    if np.sqrt((1 / (len(X) + 1)) * np.sum(np.square([f(X.iloc[i, :]) - f(x_bar) for i in range(len(X))]))) < epsilon:
      break

  # create a dataframe
  df = pd.DataFrame({"X_bar": X_bar_list,
                       "X_h": X_h_list,
                       "X_l": X_l_list,
                       "X_new": X_new_list,
                       "f(X_new)": f_new_list,
                        "type":type_list}).rename_axis('Iteration')


  df["X_new"] = df["X_new"].apply(lambda x: np.array(x))
  return df


$$
\text{Input: }
$$

$$
\alpha = 1
$$

$$
\beta = 0.25
$$

$$
γ = 4
$$

$$
ϵ = 1e-8
$$

$$
\text{3 starting points} = (3, 2), (2, 3), (0, 0)
$$


In [ ]:

points_1 = pd.DataFrame({'x1':[3,2,0], 'x2' : [2,3,0]})
simplex_search(alpha=1, beta=0.25, gamma=4, epsilon=1e-8, function=f, starting_points=points_1)

,X_bar,X_h,X_l,X_new,f(X_new),type
Iteration,,,,,,
0,"(1.0, 1.5)","(3, 2)","(0, 0)","[-1.0, 1.0]",1314.000000,R
1,"(-0.5, 0.5)","(2, 3)","(0, 0)","[0.125, 1.125]",13.453125,C
2,"(0.0625, 0.5625)","(-1.0, 1.0)","(0.125, 1.125)","[0.328125, 0.453125]",16.205582,C
3,"(0.0625, 0.5625)","(0.328125, 0.453125)","(0.125, 1.125)","[0.12890625, 0.53515625]",14.559729,C
4,"(0.126953125, 0.830078125)","(0.0, 0.0)","(0.125, 1.125)","[0.634765625, 4.150390625]",7.107344,E
5,"(0.3798828125, 2.6376953125)","(0.12890625, 0.53515625)","(0.634765625, 4.150390625)","[0.630859375, 4.740234375]",11.351156,R
6,"(0.6328125, 4.4453125)","(0.125, 1.125)","(0.634765625, 4.150390625)","[0.505859375, 3.615234375]",8.898502,C
7,"(0.5703125, 3.8828125)","(0.630859375, 4.740234375)","(0.634765625, 4.150390625)","[0.509765625, 3.025390625]",8.731363,R
8,"(0.572265625, 3.587890625)","(0.505859375, 3.615234375)","(0.634765625, 4.150390625)","[0.638671875, 3.560546875]",7.388971,R


$$
\text{Input: }
$$

$$
\alpha = 1
$$

$$
\beta = 0.5
$$

$$
γ = 2
$$

$$
ϵ = 1e-8
$$

$$
\text{3 starting points} = (2, 1), (1, 2), (0, 0)
$$


In [ ]:
pd.set_option('display.max_rows', None)
points_2 = pd.DataFrame({'x1':[2,1,0], 'x2' : [1,2,0]})
simplex_search(alpha = 1, beta = 0.5, gamma = 2, epsilon = 1e-8, function=f, starting_points=points_2)

,X_bar,X_h,X_l,X_new,f(X_new),type
Iteration,,,,,,
0,"(0.5, 1.0)","(2, 1)","(0, 0)","[-1.0, 1.0]",1314.000000,R
1,"(0.5, 1.0)","(-1, 1)","(0, 0)","[-0.25, 1.0]",40.441406,C
2,"(-0.125, 0.5)","(1.0, 2.0)","(0.0, 0.0)","[0.4375, 1.25]",13.151382,C
3,"(0.21875, 0.625)","(-0.25, 1.0)","(0.4375, 1.25)","[-0.015625, 0.8125]",15.051306,C
4,"(0.2109375, 1.03125)","(0.0, 0.0)","(0.4375, 1.25)","[0.6328125, 3.09375]",8.314539,E
5,"(0.53515625, 2.171875)","(-0.015625, 0.8125)","(0.6328125, 3.09375)","[0.259765625, 1.4921875]",12.305204,C
6,"(0.4462890625, 2.29296875)","(0.4375, 1.25)","(0.6328125, 3.09375)","[0.455078125, 3.3359375]",9.435071,R
7,"(0.5439453125, 3.21484375)","(0.259765625, 1.4921875)","(0.6328125, 3.09375)","[1.1123046875, 6.66015625]",2.036830,E
8,"(0.87255859375, 4.876953125)","(0.455078125, 3.3359375)","(1.1123046875, 6.66015625)","[1.70751953125, 7.958984375]",-2.012818,E
